# Trabajo Práctico 2

## Procesamiento del lenguaje natural

- Se importan las librerías necesarias

In [1]:
!pip install chromadb
!pip install PyPDF2
!pip install wikidataintegrator sparqlwrapper
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.7/507.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.5

In [2]:
import pprint
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
from PyPDF2 import PdfReader
from urllib.parse import quote
import os
from langchain.text_splitter import NLTKTextSplitter
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import unicodedata
import re
import tensorflow
import tensorflow_hub as hub
import chromadb
import warnings
warnings.filterwarnings("ignore")
# Descargar el conjunto de palabras de parada en español
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!pip install tensorflow-text==2.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 16.9 MB/s eta 0:00:00


In [4]:
import tensorflow_text

## Ejercicio 1 - RAG

Crear un chatbot experto en un tema a elección, usando la técnica RAG (Retrieval Augmented Generation). Como fuentes de conocimiento se utilizarán al menos las siguientes fuentes:

- Documentos de texto
- Datos numéricos en formato tabular (por ej., Dataframes, CSV, sqlite, etc.)
- Base de datos de grafos (Online o local)

El sistema debe poder llevar a cabo una conversación en lenguaje español. El usuario podrá hacer preguntas, que el chatbot intentará responder a partir de datos de algunas de sus fuentes. El asistente debe poder clasificar las preguntas, para saber qué fuentes de datos utilizar como contexto para generar una respuesta.

Requerimientos generales

- Realizar todo el proyecto en un entorno Google Colab
- El conjunto de datos debe tener al menos 100 páginas de texto y un mínimo de 3 documentos.
- Realizar split de textos usando Langchain (RecursiveTextSearch, u otros métodos disponibles). Limpiar el texto según sea conveniente.
- Realizar los embeddings que permitan vectorizar el texto y almacenarlo en una base de datos ChromaDB
- Los modelos de embeddings y LLM para generación de texto son a elección

### Tema elegido
Corte y confección de prendas, patronaje y diseño.

### 1.1 - Base de datos de vectores

En esta fuente de datos externa, se tendrá información y datos relacionados con la enseñanza de vocabulario y técnicas para aprender costura, diseño y patronaje.

#### 1.1.1 - Función para obtener el texto de los archivos fuente

La siguiente función guarda los textos redactados en los documentos en formato string en un diccionario cuyas claves son los nombres de los archivos y el valor correspondiente es el contenido del archivo.

In [62]:
def get_text_from_files(folder):
    text = {}

    for file in os.listdir(folder):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(folder, file)

            with open(pdf_path, 'rb') as pdf_file:
                pdf_reader = PdfReader(pdf_file)
                text[file] = ""
                print(f'Cantidad de páginas en el archivo\n {file}: {len(pdf_reader.pages)}')
                for page in pdf_reader.pages:
                    page_text = page.extract_text()

                    text[file] += page_text

    return text


In [60]:
# Configuración de PrettyPrinter
pp = pprint.PrettyPrinter(width=120)

In [66]:
# Ruta de la carpeta con los archivos PDF
folder_path = '/content'

# Extraer texto de los PDFs
pdf_texts = get_text_from_files(folder_path)

# Mostrar el texto extraído de los archivos PDF
pp.pprint(pdf_texts)

Streaming output truncated to the last 5000 lines.
                   '•\n'
                   ' Ubicamos la parte media de 36 cm. En el punto 0 ir amoldando lo\n'
                   ' \n'
                   'que es el cruce de botones y el armado de aletas.\n'
                   '•\n'
                   ' Del \n'
                   'punto D al 1 y del C al 2 utilizamos la medida de cruce de \n'
                   'botones que debe ser 3 cm. De la misma manera del punto B al 4 y del punto A al 3 debe ser una '
                   'medida también de 3 cm.\n'
                   '•\n'
                   ' Del punto D al d y del B al b encuadradar la línea en sentido\n'
                   ' \n'
                   'horizontal, utilizando una medida de 3,5 cm de altura.\n'
                   '•\n'
                   ' Del punto d al 7 y del punto b al 8 encuadramos una línea horizontal\n'
                   ' \n'
                   'subiendo 4 cm. de altura para el doblado del cuello.\n'
    

#### 1.1.2 - Se separan los textos en chunks

Se utiliza NLTKTextSplitter que por defecto separa por doble saltoo de línea. Con esto se busca no cortar una oración sin que haya terminado como podría pasar si se usa un chunk con cantidad de caracteres fijo. De esta manera, se busca capturar mejor el contenido semántico del texto.

In [8]:
text_splitter = NLTKTextSplitter()

docs = []
for key in pdf_texts.keys():
    docs.append(text_splitter.split_text(pdf_texts[key]))

print(f'Amount of splitted documents: {len(docs)}')
print('Documento splitteado (lista de chunks del primer documento)')
pprint.pprint(docs[0][0:3])

Amount of splitted documents: 3
Documento splitteado (lista de chunks del primer documento)
['TEXTO GUIA DEL PARTICIPANTE\n'
 'CONFECCIÓN TEXTIL\n'
 'TEXTO GUIA DEL PARTICIPANTE\n'
 'CONFECCIÓN TEXTIL\n'
 'Texto Guía del ParticipanteCooperación Suiza en Bolivia\n'
 'Proyecto:\n'
 'Formación técnica profesional - FTPFundación Educación para el Desarrollo '
 'FAUTAPO   \n'
 'Coordinador Regional FAUTAPO Chuquisaca:\n'
 'Ing.\n'
 '\n'
 'Marco Antonio Flores Gárate\n'
 'Responsable Técnico:\n'
 'Lic.\n'
 '\n'
 'Filomeno Cerezo Chacón\n'
 'Equipo técnico proyecto Formación técnica profesional:\n'
 'Ing.\n'
 '\n'
 'Favio Raya Castro Ing.\n'
 '\n'
 'Gisela Parra MartínezLic.\n'
 '\n'
 'María Elena Cuellar Mina Lic.\n'
 '\n'
 'Margoth Zulema Miranda CarballoTéc.\n'
 '\n'
 'Grover Araujo Marín Téc.\n'
 '\n'
 'Iván Espada SotoIng.\n'
 '\n'
 'Lino FloresIng.\n'
 '\n'
 'Reynaldo Panoso CaeroIng.\n'
 '\n'
 'Edwin Peñaranda Iporre\n'
 'El presente Texto guía,  fue elaborado por el Grover Araujo Marí

#### 1.1.3 - Preprocesamiento de los datos

Se han definido distintas funciones que se utilizarán para preprocesar el texto que será vectorizado luego, buscando con las mismas obtener un texto más enfocado en los significados intrínsecos de las oraciones y el contexto y con menos ruido.

- Función para remover acentos

La eliminación de acentos simplifica el texto y evita posibles problemas de codificación que podrían surgir con caracteres acentuados.

In [9]:
def remove_accents(text):
    nfkd_form = unicodedata.normalize('NFKD', text)
    return ''.join([part for part in nfkd_form if not unicodedata.combining(part)])

- Función para reemplazar palabras

Se reemplaza una abreviatura muy común en el ámbito de la costura por su palabra completa para que sea semánticamente más destacable. Además, se extienen palabras que tienen sinónimos utilizados en distintos lugares del mundo en español para que formen parte de los embeddings que brindarán información al modelo.

In [10]:
# Se define una función para eliminar las stopwords de una frase
def change_words(text):
    text = text.replace('falda', 'falda/pollera')
    text = text.replace('faldas', 'falda/pollera')
    text = text.replace('blusa', 'blusa/remera/camiseta')
    text = text.replace('blusas', 'blusa/remera/camiseta')
    text = text.replace('prenda', 'prenda/ropa')
    text = text.replace('prendas', 'prenda/ropa')
    text = text.replace(' cm ', 'centimetro')
    text = text.replace(' cm.', 'centimetro')
    # Remover números
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\b\d+', '', text)
    text = re.sub(r'\d+\b', '', text)
    text = re.sub(r'\d+', '', text)

    return text

- Función para remover los mails y urls del texto

Se remueven porque se considera que no aportan a la enseñanza de técnicas y conceptos para el tema propuesto.

In [11]:
def replace_url_and_mail(text):
    # Definir el patrón para encontrar una URL que comienza con www
    patron_url = r'www\.[^\s]+'

    # Patrón de expresión regular para encontrar direcciones de correo electrónico
    patron_mail = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

    # Utilizar re.sub para reemplazar direcciones de correo electrónico con un espacio vacío
    text = re.sub(patron_mail, '', text)

    # Utilizar re.sub para reemplazar el patrón con un text vacío
    text_modified = re.sub(patron_url, '', text)

    return text_modified

- Función para remover stopwords

Las stopwords son palabras comunes como "el", "la", "y", etc., que no aportan mucha información en términos de contexto y significado. Se eliminan porque ayuda a reducir la dimensionalidad del texto y a concentrarse en las palabras clave.

In [12]:
# Definir las palabras de parada en español
stop_words = set(stopwords.words('spanish'))

In [13]:
def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.casefold() not in stop_words]
    return " ".join(filtered_text)

- Función que engloba las anteriores para ser utilizada sobre el texto

In [14]:
def text_processing(doc):
    for i, chunk in enumerate(doc):
       # Convertir a minúsculas
       chunk = chunk.lower()
       # Remover urls
       chunk = replace_url_and_mail(chunk)
       # Remover puntuación
       chunk = re.sub(r'[^\w\s]', '', chunk)
       # Remover stopwords
       chunk = remove_stopwords(chunk)
       # Remover acentos
       chunk = remove_accents(chunk)
       # Ampliar palabras
       chunk = change_words(chunk)
       doc[i] = chunk
    return doc


Para cada chunk de texto, se aplica el preprocesamiento utilizando las funciones anteriores y luego se muestra el resultado.

In [15]:
for doc in docs:
    doc = text_processing(doc)

In [16]:
pprint.pprint(docs[0][0:3])

['texto guia participante confeccion textil texto guia participante confeccion '
 'textil texto guia participantecooperacion suiza bolivia proyecto formacion '
 'tecnica profesional ftpfundacion educacion desarrollo fautapo coordinador '
 'regional fautapo chuquisaca ing marco antonio flores garate responsable '
 'tecnico lic filomeno cerezo chacon equipo tecnico proyecto formacion tecnica '
 'profesional ing favio raya castro ing gisela parra martinezlic maria elena '
 'cuellar mina lic margoth zulema miranda carballotec grover araujo marin tec '
 'ivan espada sotoing lino floresing reynaldo panoso caeroing edwin penaranda '
 'iporre presente texto guia elaborado grover araujo marin marco proyecto '
 'formacion tecnica profesional ejecutado fundacion educacion desarrollo '
 'fautapo financiamiento cooperacion suiza modulo ii texto referido patronaje '
 'corresponde tec javier patzy calani sido enriquecido adaptado niveles '
 'formacion tecnico basico tecnico auxiliar carrera confeccio

#### 1.1.4 - Vactorización del texto

Se utiliza un modelo de embeddings para vectorizar el texto ya que tiene en cuenta la semántica y coontexto de lo que representa. Se busca así, robustecer la representación del texto al pasarlo a vectores.

In [17]:
# El modelo que usaremos es uno multilingüe (de 16 idiomas)
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3'
embed = hub.load(module_url)

In [67]:
# Función para realizar el embedding del input
def embed_text(input):
    return embed(input)

In [68]:
# Configuración inicial de ChromaDB, la base de datos vectorial a utilizar
client = chromadb.Client()
collection = client.get_or_create_collection("libros_aprender_corte_y_confeccion_y_patronaje")


In [20]:
# Convertir las claves del diccionario a una lista
book_names_list = list(pdf_texts.keys())

El siguiente bloque de código añade a la colección de la base de datos vectorial Chromadb los fragmentos de texto convertidos a vectores.

Luuego, realiza una consulta para conocer la respuesta de la base de datos a la misma.

In [69]:
for i, doc in enumerate(docs):
    book_name = book_names_list[i]
    ids_textos = [f"libro{book_name}parte{i}" for i in range(1, len(doc)+1)]
    fuentes = [f"{book_name}" for i in range(1, len(doc)+1)] # Ejemplo de metadatos
    # Calcular embeddings para los documentos
    embeddings = embed(doc).numpy().tolist() # Convertir a lista para que sea serializable
    collection.add(
        documents=doc,
        metadatas=[{"source": fuente} for fuente in fuentes],
        ids=ids_textos,
        embeddings=embeddings
        )
# Realizar una consulta
consulta = "Cómo se cose una remera?"
embedding_consulta = embed([consulta]).numpy().tolist()
results = collection.query(
            query_embeddings=embedding_consulta, # Aquí pasamos el embedding de la consulta
            n_results=3 # Traemos los 3 resultados más cercanos
            )
# Imprimir los documentos y sus distancias
for id, doc, distancia in zip(results["ids"], results["documents"], results["distances"]):
            print(f"IDs: {id}")
            print(f"Documento: {doc}")
            print(f"Distancia: {distancia}")
            print('-' * 40)

IDs: ['libroBLUSASBAJA.pdfparte9', 'libroBLUSASBAJA.pdfparte14', 'libroBLUSASBAJA.pdfparte6']
("Documento: [' demostracion grafica mangafig  fig   medidas largo manga centimetrocontorno brazo centimetrocontorno "
 'muneca puno centimetrocontorno sisacentimetromedidas manga toman acuerdo siguiente detalle largo manga pasa metro '
 'nudo pronunciado brazo aproximandose muneca mano indica foto  contorno brazo debe pasar metro alrededor brazo '
 'horizontalmente parte ensanchada debe tomarse tal manera flojo tampoco ajustado contorno puno medida necesario pasar '
 'metro alrededor puno horizontalmente tambien haciendo puno dedos bordeamos alrededor ello   como toman medidas manga '
 'fig  foto  foto  foto medidas auxiliares medidas calcula ayuda medidas principales constantes pueden encontrar '
 'respectivas tablas proporciones segun caso seguir correctamente cualquier indicacion costura lograr mejores '
 'confecciones usted debe tomar cuenta tallas indicadas si medidas corporales dos tall

### 1.2 - Base de datos de grafos

Esta fuente de datos externa será utilizada para abarcar las búsquedas de personalidades históricas relevantes en el ámbito de la costura y la moda.

La base de datos de grafos que se utilizará es dbpedia.

Se descarga el paquete en español

In [22]:
!python -m spacy download es_core_news_lg

2023-12-18 20:43:46.374442: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 20:43:46.374514: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 20:43:46.375879: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-18 20:43:47.643461: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


Se definen las distintas funciones que permitirán la interacción con la base de datos de grafos. En órden de aparición:

- Función para realizar la consulta a la base de datos de grafos
- Función para obtener el nombre de la personalidad por la que se consulta:
  Esta función analiza la query en busca de NER (entidades nombradas) y las extrae de la misma. Retornando una lista de los nombres hallados.
- Función que arma la la consulta en lenguaje SPARQL para que pueda ser entendida por la base de datos de grafos y luego, realiza la consulta a la misma.

Se muestra luego un ejemplo de uso y los resultados obtenidos

In [23]:
import spacy
from SPARQLWrapper import SPARQLWrapper, JSON

def ejecutar_consulta_sparql(consulta):
    sparql = SPARQLWrapper("http://es.dbpedia.org/sparql")
    sparql.setQuery(consulta)
    sparql.setReturnFormat(JSON)
    resultados = sparql.query().convert()
    return resultados

def extraer_nombre_disenador(oracion):
    # Cargar el modelo de procesamiento de spaCy para el idioma español
    nlp = spacy.load("es_core_news_lg")

    # Procesar la oración
    doc = nlp(oracion)
    # Imprimir las entidades nombradas, etiquetas y explicaciones
    for ent in doc.ents:
      print(f'Entidad: {ent.text}, Etiqueta: {ent.label_}, Explicación: {spacy.explain(ent.label_)}')
    # Encontrar entidades nombradas (por ejemplo, nombres de personas)
    nombres_personas = [ent.text for ent in doc.ents if ent.label_ == "PER"]

    # Supongamos que el nombre del diseñador es el primer nombre de persona encontrado
    nombre_disenador = nombres_personas if nombres_personas else None

    return nombre_disenador

def consultar_dbpedia_por_disenador(nombre_disenador):
    consulta_sparql = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?s ?abstract ?tipo
    WHERE {{
        ?s rdfs:label ?label.
        FILTER regex(?label, "{nombre_disenador}", "i").
        FILTER (lang(?label) = "es").
        OPTIONAL {{ ?s dbo:abstract ?abstract. FILTER (lang(?abstract) = "es") }}
        OPTIONAL {{ ?s rdf:type ?tipo }}
    }}
    LIMIT 1
    """

    resultados = ejecutar_consulta_sparql(consulta_sparql)

    if resultados["results"]["bindings"]:
        disenador_uri = resultados["results"]["bindings"][0].get("s", {}).get("value", "")
        abstract = resultados["results"]["bindings"][0].get("abstract", {}).get("value", "")

        return disenador_uri, abstract
    else:
        return None, f"No se encontró información sobre el diseñador {nombre_disenador} en DBpedia."

# Ejemplo de uso
consulta_usuario = "Busca información sobre la diseñadora de moda Giorgio Armani y Carolina Herrera."
nombre_disenador = extraer_nombre_disenador(consulta_usuario)
def get_answer(nombre_disenador):
  if nombre_disenador:
    for disenador in nombre_disenador:
        respuesta_info = ''
        disenador_uri, respuesta_dbpedia = consultar_dbpedia_por_disenador(disenador)
        if disenador_uri:
            print(f"URI del diseñador {disenador}: {disenador_uri}")
            print(f"Información sobre {disenador} en DBpedia:")
            print(respuesta_dbpedia)
            respuesta_info += respuesta_dbpedia + '\n'
        else:
            print(f"No se encontró información sobre el diseñador {disenador} en DBpedia.")
  else:
      print("No se pudo extraer un nombre de diseñador de la consulta del usuario.")
  return respuesta_info

respuesta_info = get_answer(nombre_disenador)

Entidad: Giorgio Armani, Etiqueta: PER, Explicación: Named person or family.
Entidad: Carolina Herrera, Etiqueta: PER, Explicación: Named person or family.
URI del diseñador Giorgio Armani: http://es.dbpedia.org/resource/Giorgio_Armani
Información sobre Giorgio Armani en DBpedia:
Giorgio Armani (Piacenza, 11 de julio de 1934) es un diseñador de moda y empresario de origen armenio-italiano. Principalmente conocido por su ropa de hombre. Formó su propia empresa en 1974 y 37 años más tarde la revista Forbes lo consideró el diseñador italiano de mayor éxito comercial, estimando su fortuna en 9,8 miles de millones de dólares. Admirado y promocionado por diferentes actores de cine y varios músicos, Armani ha diversificado su producción con perfumería y otros complementos en lo que se conoce como el Emporio Armani.
URI del diseñador Carolina Herrera: http://dbpedia.org/resource/Carolina_Herrera
Información sobre Carolina Herrera en DBpedia:
María Carolina Josefina Pacanins Niño (Caracas; 8 de

### 1.3 - Datos en formato tabular

En esta fuente de datos externa, se cuent con los datos numéricos que corresponden a las medidas de talles para distintas secciones del cuerpo. Será consultada en caso de que se solicite información relacionada a obtener mediciones de referencia.

Se leen los archivos tabulares y se convierten a un string que conserva su estructura tabular.

In [31]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
df_tallas_ninios = pd.read_csv('TALLAS_DE_NIÑOS_AS_EN_CENTIMETROS.csv')

# Convertir el DataFrame a un string legible
tallas_ninios = f'TALLAS_DE_NIÑOS_AS_EN_CENTIMETROS\n {df_tallas_ninios.to_string(index=False)}'

# Cargar el archivo CSV en un DataFrame
df_tallas_hombre = pd.read_csv('TABLA_DE_MEDIDAS_MASCULINAS.csv')

# Convertir el DataFrame a un string legible
tallas_hombre = f'TABLA_DE_MEDIDAS_MASCULINAS_EN_CENTIMETROS\n {df_tallas_hombre.to_string(index=False)}'

# Cargar el archivo CSV en un DataFrame
df_tallas_mujer = pd.read_csv('TABLA_DE_MEDIDAS_FEMENINAS.csv')

# Convertir el DataFrame a un string legible
tallas_mujer = f'TABLA_DE_MEDIDAS_FEMENINAS_EN_CENTIMETROS\n {df_tallas_mujer.to_string(index=False)}'

# 'tallas_ninios' ahora contiene el contenido del CSV en forma de string legible
print(tallas_ninios)

# 'tallas_hombre' ahora contiene el contenido del CSV en forma de string legible
print(tallas_hombre)

# 'tallas_mujer' ahora contiene el contenido del CSV en forma de string legible
print(tallas_mujer)


TALLAS_DE_NIÑOS_AS_EN_CENTIMETROS
               EDAD     2     4    6    8   10   12   14
 CONTORNO DE CUELLO 24,5  26,5 28,5 30,5 32,5 34,5 36,5
 CONTORNO DE PECHO    54    58   62   66   72   76   80
CONTORNO DE CINTURA   50    53   56   59   61   63   65
CONTORNO DE CADERA    58    62   66   70   74   78   82
    LARGO DE TALLE  18,5  21,5 24,5 27,5 29,5 31,5   33
     LARGO A CADERA   10  11,5   13 14,5   16 17,5   19
    LARGO A RODILLA 32,5 36,75 39,5   43 46,5   50 53,5
       LARGO HOMBRO  7,2     8  8,7  9,3  9,9 10,5   11
     LARGO DE BRAZO   30    34   38   42   46   50   54
     LARGO PANTALON   56    62   68   74   80   86   92
           ESTATURA   92   102  112  122  132  142  152
TABLA_DE_MEDIDAS_MASCULINAS_EN_CENTIMETROS
 TOMAR MEDIDAS EN      SECTOR  TALLA S (38) HOMBRE TALLA S (40) HOMBRE  TALLA M (42) HOMBRE TALLA M (44) HOMBRE  TALLA L (46) HOMBRE TALLA L (48) HOMBRE  TALLA XL (50) HOMBRE TALLA XL (52) HOMBRE
          SECTOR ABREVIATURA                   38     

### 1.4 - Elección de la fuente externa para la consulta

A continuación, se requiere definir un criterio para determinar la fuente que resulte más apta para proveer los datos de contexto que permiten una respuesta más precisa y exacta de la consulta.

Se utiliza un modelo liviano para realizar esta elección.
Inicialmente, se definen ejemplos de consultas que coinciden con cada fuente externa en la variable few_shots_examples.

Se toma la consulta del usuario, y se calcula la similaridad de dicha consulta con las consultas de ejemplo de las distintas fuentes. La fuente con la que se encuentre mayor similaridad, es la seleccionada como la más apta. Para ser seleccionada, además, debe superar un umbrl de similaridad. Con esto se busca que consultas muy alejadas de las propuestas como patrones, no sean consideradas a ser contextualizadas con las fuents externas.

In [24]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=ee96bcca44a2ebd6b7dbcc7adeb2d408ceb5105e3260e55fc11b45c89a1fc196
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [47]:
from sentence_transformers import SentenceTransformer, util

# Cargar el modelo sentence-transformers
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Ejemplos de pocos disparos para cada fuente de datos
few_shot_examples = {
    'grafo': ['Consulta sobre diseñadores famosos.', 'Giorgio Armani', 'Carolina Herrera', 'Quién fue Coco Chanel?', 'Por qué es famoso el diseñador o diseñadora Valentino?'],
    'csv': ['Talles de prendas en un archivo CSV.', 'Cómo está organizada la tabla de talles.', 'Cuáles son las medidas de los talles de remera o blusa para mujer, niña, niño, hombre.',
            'Números de talles personas', 'Medidas de ropa para mujer'],
    'vectores': ['Embeddings de texto sobre corte y confección.', 'Qué información contiene la base de datos de vectores.', 'Cómo coser una blusa o remera o camiseta',
                 'Aprender costura','Máquina de coser', 'Corte y confección', 'Patrones de vestidos', 'Diseño patronaje',
                 'Aquí se toman consultas exclusivamente sobre patronaje, costura, corte y confeccion de prendas de ropa', 'Costura para principiantes']
}

def few_shot_learning(query, few_shot_examples):
    # Obtener representación embebida de la consulta del usuario
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Obtener representaciones embebidas de los ejemplos de pocos disparos
    examples_embeddings = {}
    for source, examples in few_shot_examples.items():
        examples_embeddings[source] = model.encode(examples, convert_to_tensor=True)

    # Calcular similitud coseno entre la consulta y cada conjunto de ejemplos
    similarities = {}
    for source, embeddings in examples_embeddings.items():
        similarities[source] = util.pytorch_cos_sim(query_embedding, embeddings).mean().item()

    # Devolver la fuente de datos con la similitud más alta
    predicted_source = max(similarities, key=similarities.get)
    if similarities[predicted_source] < 0.4:
      predicted_source = 'No se ha detectado que la consulta pueda ser respondida por las distintas fuentes externas'
    return predicted_source

In [26]:
# Ejemplo de uso
consulta_usuario1 = "Cómo coser un pantalón?"
consulta_usuario2 = "Cuáles son las medidas de talles de pantalón para mujer?"
consulta_usuario3 = "Quién fue Gianni Versace?"
fuente_seleccionada1 = few_shot_learning(consulta_usuario1, few_shot_examples)
fuente_seleccionada2 = few_shot_learning(consulta_usuario2, few_shot_examples)
fuente_seleccionada3 = few_shot_learning(consulta_usuario3, few_shot_examples)
print(f"La fuente de datos seleccionada para la consulta {consulta_usuario1} es: {fuente_seleccionada1} y se esperaba que fuera vectores")
if fuente_seleccionada1 == 'vectores':
  print('Se ha seleccionado la fuente esperada')
else:
  print('No se ha seleccionado la fuente esperada')
print('\n')
print(f"La fuente de datos seleccionada para la consulta {consulta_usuario2} es: {fuente_seleccionada2} y se esperaba que fuera csv")
if fuente_seleccionada2 == 'csv':
  print('Se ha seleccionado la fuente esperada')
else:
  print('No se ha seleccionado la fuente esperada')
print('\n')
print(f"La fuente de datos seleccionada para la consulta {consulta_usuario3} es: {fuente_seleccionada3} y se esperaba que fuera grafo")
if fuente_seleccionada3 == 'grafo':
  print('Se ha seleccionado la fuente esperada')
else:
  print('No se ha seleccionado la fuente esperada')
print('\n')

La fuente de datos seleccionada para la consulta Cómo coser un pantalón? es: vectores y se esperaba que fuera vectores
Se ha seleccionado la fuente esperada


La fuente de datos seleccionada para la consulta Cuáles son las medidas de talles de pantalón para mujer? es: csv y se esperaba que fuera csv
Se ha seleccionado la fuente esperada


La fuente de datos seleccionada para la consulta Quién fue Gianni Versace? es: grafo y se esperaba que fuera grafo
Se ha seleccionado la fuente esperada




### 1.5 - RAG (Generación Aumentada por Recuperación)

Los LLMs de última generación son entrenados con grandes cantidades de datos para lograr un amplio espectro de conocimiento general almacenado en los pesos de la red neuronal (memoria paramétrica). Sin embargo, al solicitar a un LLM que genere una respuesta que requiere conocimientos que no estaban incluidos en sus datos de entrenamiento, como información más reciente, propietaria o específica de un dominio, puede llevar a inexactitudes fácticas, o bien puede no conocer la respuesta.

La Generación Aumentada por Recuperación (RAG) es el concepto de proporcionar a los Modelos de Lenguaje de Gran Escala (LLMs) información adicional proveniente de una fuente de conocimiento externa. Esto les permite generar respuestas más precisas y contextuales.

En este caso, se proporcionará a al LLM zephyr-7b-beta, información de contexto adicional para la temática de 'Corte y confección de prendas, patronaje y diseño'.

Una vez definidas las funciones y fuentes precisadas para dar contexto al modelo LLM se definen las funciones pertinentes para implementar el RAG y realizar las consultas al mismo.

In [27]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from jinja2 import Template

- Función para realizar la request al modelo LLM

In [43]:
# Aquí hacemos la llamada el modelo
def generate_answer(prompt: str, max_new_tokens: int = 768) -> None:
  try:
    # Tu clave API de Hugging Face
    api_key = 'YOUR_HUGGINGFACE_KEY'
    # URL de la API de Hugging Face para la generación de texto
    api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"
    # Cabeceras para la solicitud
    headers = None
    if api_key != '':
      headers = {"Authorization": f"Bearer {api_key}"}
    # Datos para enviar en la solicitud POST
    # Sobre los parámetros: https://huggingface.co/docs/transformers/main_classes/text_generation
    data = {
    "inputs": prompt,
    "parameters": {
    "max_new_tokens": max_new_tokens,
    "temperature": 0.5,
    "top_k": 15,
    "top_p": 0.95
    }
    }
    if headers is not None:
      # Realizamos la solicitud POST
      response = requests.post(api_url, headers=headers, json=data)
      # Extraer respuesta
      respuesta = response.json()[0]["generated_text"][len(prompt):]
      return respuesta
  except Exception as e:
    print(f"An error occurred: {e}")

- Función que tiene las instrucciónes de formación del prompt que se envía al modelo para realizar la consulta

In [29]:
def zephyr_instruct_template(messages, add_generation_prompt=True):
  # Definir la plantilla Jinja
  template_str = "{% for message in messages %}"
  template_str += "{% if message['role'] == 'user' %}"
  template_str += "<|user|>{{ message['content'] }}</s>\n"
  template_str += "{% elif message['role'] == 'assistant' %}"
  template_str += "<|assistant|>{{ message['content'] }}</s>\n"
  template_str += "{% elif message['role'] == 'system' %}"
  template_str += "<|system|>{{ message['content'] }}</s>\n"
  template_str += "{% else %}"
  template_str += "<|unknown|>{{ message['content'] }}</s>\n"
  template_str += "{% endif %}"
  template_str += "{% endfor %}"
  template_str += "{% if add_generation_prompt %}"
  template_str += "<|assistant|>\n"
  template_str += "{% endif %}"
  # Crear un objeto de plantilla con la cadena de plantilla
  template = Template(template_str)

  # Renderizar la plantilla con los mensajes proporcionados
  return template.render(messages=messages, add_generation_prompt=add_generation_prompt)

- Función que adiciona a la consulta el contexto brindado por la fuente externa y los transforma en el prompt específico de dicha consulta para procesar por el modelo.

In [30]:
# Esta función prepara el prompt en estilo QA
def prepare_prompt(query_str: str, context_info: list):
  TEXT_QA_PROMPT_TMPL = (
  "La información de contexto es la siguiente:\n"
  "---------------------\n"
  "{context_str}\n"
  "---------------------\n"
  "Dada la información de contexto anterior, y sin utilizar conocimiento previo, responde en español la siguiente pregunta.\n"
  "Pregunta: {query_str}\n"
  "Respuesta: "
  )
  # Construimos el contexto de la pregunta
  context_str = context_info
  messages = [
  {
  "role": "system",
  "content": "Eres un asistente útil que siempre responde con respuestas veraces, útiles y basadas en hechos.",
  },
  {"role": "user", "content": TEXT_QA_PROMPT_TMPL.format(context_str=context_str, query_str=query_str)},
  ]
  final_prompt = zephyr_instruct_template(messages)
  return final_prompt

- Función que determina el contexto a adicionar en la consulta del usuario en base a la categoría de fuente externa recibida puede ser que se reciba como parámetro que no es adecuado adicionar contexto de alguna de las tres fuentes para la consulta realizada. Esto puede ocurrir si se realizan consultas muy alejadas de la temática a abordar de manera más precisa.

In [32]:
def context_info_from_external_source(source, query):
  context_info = ''
  if source == 'csv':
    # Verificar si la palabra está presente en la cadena
    if 'niñ' in query:
      context_info = tallas_ninios
    elif 'mujer' in query or 'femenin' in query:
      context_info = tallas_mujer
    elif 'hombre' in query or 'masculin' in query:
      context_info = tallas_hombre
    else:
      context_info = f'{tallas_ninios}\n\n {tallas_mujer}\n\n {tallas_hombre}'
  elif source == 'grafo':
    designer_name = extraer_nombre_disenador(query)
    for designer in designer_name:
      _, info_dbpedia = consultar_dbpedia_por_disenador(designer)
      context_info += info_dbpedia + '\n'
  elif source == 'vectores':
    embedding_consulta = embed([query]).numpy().tolist()
    results = collection.query(
                query_embeddings=embedding_consulta, # Aquí pasamos el embedding de la consulta
                n_results=3 # Traemos los 3 resultados más cercanos
                )
    if results['documents']:
      for result in results['documents']:
        for text in result:
          context_info += text + '\n'
  else:
    context_info = 'No se ha detectado una fuente de contexto para la consulta realizada'
  return context_info

- Función que analiza la query y devuelve la respuesta encontrada haciendo uso de las funciones anteriores

In [34]:
print('Realizando llamada a HuggingFace para generar respuestas...\n')
queries = ['Quién fue Coco Chanel?', 'Aprender costura y patronaje', 'Cuáles son las medidas de talles de las prendas de ropa?', 'Quiero recetas de postres con chocolate']

def procesar_pregunta(queries):
  for query_str in queries:
    # Traemos los documentos más relevantes para la consulta
    source_for_context = few_shot_learning(query_str, few_shot_examples)
    print(f'Selected context source: {source_for_context}')
    context_text = context_info_from_external_source(source_for_context, query_str)
    final_prompt = None
    if source_for_context == 'No se ha detectado que la consulta pueda ser respondida por las distintas fuentes externas':
      final_prompt = prepare_prompt(f'Dar la respuesta en el idioma en que fue realizada la pregunta. Responder: {query_str}', 'No hay contexto adicional para esta consulta')
    if source_for_context == 'csv':
      query_str_csv = f'Retorna en la respuesta todas las tablas de talles que te fueron pasadas como contexto sin realizarles ninguna modificacion.\n {query_str}'
      final_prompt = prepare_prompt(query_str_csv, str(context_text))
    else:
      final_prompt = prepare_prompt(query_str, str(context_text))
    print('Pregunta:', query_str)
    print('Respuesta:')
    print(generate_answer(final_prompt))
    print('-------------------------------------------------------')
  return 'queries answered'

_ = procesar_pregunta(queries)

Realizando llamada a HuggingFace para generar respuestas...

Selected context source: grafo
Entidad: Coco Chanel, Etiqueta: PER, Explicación: Named person or family.
Pregunta: Quién fue Coco Chanel?
Respuesta:
Coco Chanel, cuyo nombre real era Gabrielle Chanel, fue una diseñadora de moda francesa de gran influencia en el siglo XX. Nació en Saumur, Francia, en 1883 y se consolidó como una de las diseñadoras más importantes y innovadoras de la historia, especialmente durante la Primera Guerra Mundial. Criada en un orfanato de monjas, fue conocida por su fuerte determinación, ambición y vitalidad, que aplicaba tanto a su vida profesional como a su vida social. Alcanzó éxito como empresaria y prominencia social en la década de 1910 gracias a los contactos que le ofrecía su trabajo. Su famoso traje sastre femenino de tweed ribeteado se convirtió en un icono de la elegancia femenina, y su perfume Chanel Nº5 es un producto mundialmente conocido. Sin embargo, su colaboración con la Gestapo dur

- Función que realiza lo mismo que la función global anterior de manera interactiva con el usuario

In [48]:
def ask_queries():
  consulta = input("Escriba su consulta: ")
  consulta_list = [consulta]
  _ = procesar_pregunta(consulta_list)
  if _:
    return _

query_input = 1
while query_input == 1:
    _ = ask_queries()
    if _ :
      otra_consulta = input("Desea realizar otra consulta? Si desea realizar otra consulta escriba si: ")
      if otra_consulta.lower() != 'si' and otra_consulta.lower() != 'sí':
        print('Si tienes otra consulta no dudes en preguntar')
        query_input = 0

Escriba su consulta: Receta de alfajores
Selected context source: No se ha detectado que la consulta pueda ser respondida por las distintas fuentes externas
Pregunta: Receta de alfajores
Respuesta:
Ingredientes:
- 250 gramos de harina
- 150 gramos de azúcar
- 100 gramos de mantequilla
- 100 gramos de miel
- 100 gramos de semillas de jazmínez
- 1 cucharada de canela molida
- 1 cucharada de vainilla extracto
- 150 ml de leche
- Azúcar para espolvorear

Instrucciones:
1. En un tazón, calienta la leche hasta el punto de ebullición.
2. Retira de la fuente de calor y agrega la miel, mezcla hasta que se disuelva.
3. Deja enfriar la mezcla hasta que tenga una temperatura de 40-45 grados Celsius.
4. En un recipiente grande, mezcle la harina, el azúcar, la mantequilla, la semilla de jazmínez, la canela molida y la vainilla extracto.
5. Agrega la mezcla de leche y miel a la mezcla de harina y mezcle hasta que se forme una masa suave.
6. Deja reposar la masa durante 30 minutos en el recipiente.
7.

Se observa que asigna la consulta a la fuente esperada y, que cuando no encuentra que corresponda asignar una de las tres fuentes para dicha consulta, responde sin añadir contexto específico con lo que devuelve el LLM para la consulta.

### Conclusiones


El proceso de generación de un chatbot experto utilizando RAG ha implicado una combinación de preprocesamiento de datos, selección y almacenamiento de información relevante, implementación de modelos para selección de fuentes de contexto y modelos de generación y recuperación.

Junto a ello, se ha realizado posteriormente una evaluación iterativa del desempeño del chatbot mediante análisis de las respuestas brindadas bajo los parámetros definidos, tales como los enviados en la request POST al modelo LLM (temperature, top p, top k), el umbral establecido para determinar la fuente externa a utilizar, el número máximo de resultados solicitados la la base de datos de vectores y la base de datos de grafos.

 Las consecutivas consultas realizadas al modelo se han encaminado en el objetivo de hallar el equilibrio entre obtener una respuesta coherente y contextualizada, el tiempo de espera de la respuesta y el uso de recursos de cómputo. Se ha encontrado que es clave iterar y ajustar continuamente el sistema para mejorar su capacidad de respuesta y precisión.